# Baseball Home Run Predictions for Y2021

# Diedrich Swetlik and JP Richmond

## Introduction

We are going to attempt to accurately predict the amount of home runs that will occur for the MLB 2021 Season. Since the season has already been completed and thus all the home runs that can occur in the 2021 season have already happened, we have a fine measurement as to just how accurate our model will be.

We are using data from the 2000-2020 seasons on players hitting stats organized by year. This dataset was obtained from http://www.seanlahman.com/baseball-archive/statistics/ using the 2020 - comma delimited version. This gives a mountain of different datasets, from which we pulled the Batting.csv dataset. The Batting.csv dataset had data on batters starting in the year 1871, which we have determined isn't exactly helpful when trying to determine more modern baseball statistics, thus our version of Batting.csv found at https://github.com/dswetlik/BaseballHRPrediction/blob/master/Batting.csv has been cut down from 1.5 centuries to 2 decades. Finally, the data that we will be testing our model with comes from https://www.baseball-reference.com/leagues/majors/2021.shtml#teams_standard_batting. We only pulled from that site the final values of the year and the columns we will need at the end. The edited version of the data can be found at https://github.com/dswetlik/BaseballHRPrediction/blob/master/2021Batting.csv.

That being said, we need to clean up our dataset a little bit more before we begin. This is because we need to compare yearly overall statistics and not individual player data. We will combine all of the statistics from every year and then begin fitting that to models.

Our process for modeling will be fairly straightforward; we will do an ordinary least squares fit utilizing all of the predictors to get the p-values. After that, we will determine collinearity values using a VIF analysis. Then, we will perform subset selection using forward subset selection. Next, we will perform Leave One Out Cross Validation to test the best polynomial degree to fit our model with and test our model's overall accuracy. Finally, we will use that model to predict the amount of home runs for the 2021 season and assess our final accuracy.

An initial disclaimer, the 2020 season was drastically shorter than every other season in the dataset due to the global pandemic and therefore the sample size is much smaller. However, we expect that the decrease in home runs and games played will also be reflected in the other predictors proportionately so that it can still be reliably used as data.

## Data Setup

In [ ]:
# Basics and Plotting
import pandas as pd
import numpy as np
import scipy as scp
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import seaborn as sns
from itertools import chain, combinations

# Sklearn Models
import sklearn.linear_model as skl_lm
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.linear_model import Lasso, Ridge
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, LeaveOneOut, KFold, cross_val_score, cross_validate
from sklearn.preprocessing import PolynomialFeatures
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import PLSRegression

# Alternative models
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.formula.api as smf

In [ ]:
baseball = pd.read_csv("https://raw.githubusercontent.com/dswetlik/BaseballHRPrediction/master/Batting.csv")

In [ ]:
baseball

Below we are dropping the columns playerID, teamID, stint, and lgID, as we have decided that they would be inconsequential or irrelavent for determining league-wide home run counts. This is because they are not even actual hitting statistics.

We are also going to drop CS, SH, SF, GIDP, SB, and HBP because they are stats related to groundouts/flyouts, at-bats that ended abruptly, and baserunning stats. Therefore, we determined that correlation would mean nothing because the stats have no real-life connection to home runs and any connection would prove to be a spurious relationship.

In [ ]:
baseball.drop(columns=["playerID","teamID","stint","lgID","CS","SH","SF","GIDP","SB","HBP"], axis=1, inplace=True)
baseball.rename(columns={"2B": "Double", "3B": "Triple"}, inplace=True)
baseball.head()

After dropping those columns, the remaining columns are as follows:


| Num | ID     | Name                       |
|-----|--------|----------------------------|
|  0  | yearID | Year                       |
|  1  | G      | Games                      |
|  2  | AB     | At Bats                    |
|  3  | R      | Runs                       |
|  4  | H      | Hits                       |
|  5  | Double | Doubles                    |
|  6  | Triple | Triples                    |
|  7  | HR     | Home Runs                  |
|  8  | RBI    | Runs Batted In             |
|  9  | BB     | Base on Balls              |
| 10  | SO     | Strikeouts                 |
| 11  | IBB    | Intentional Walks          |

This is almost usable for what we want, but it is still organized per-player, and we want it to be based on the year's total statistics. We will go through and create a new dataset now based on years.

In [ ]:
baseballYearTotal = []
for i in range(2000,2021):
    baseballYear = baseball.loc[baseball['yearID'] == i].to_dict(orient='dict')
    G = 0
    for j in baseballYear['G'].values():
        G += j
    AB = 0
    for j in baseballYear['AB'].values():
        AB += j
    R = 0
    for j in baseballYear['R'].values():
        R += j
    H = 0
    for j in baseballYear['H'].values():
        H += j
    Double = 0
    for j in baseballYear['Double'].values():
        Double += j
    Triple = 0
    for j in baseballYear['Triple'].values():
        Triple += j
    HR = 0
    for j in baseballYear['HR'].values():
        HR += j
    RBI = 0
    for j in baseballYear['RBI'].values():
        RBI += j
    BB = 0
    for j in baseballYear['BB'].values():
        BB += j
    SO = 0
    for j in baseballYear['SO'].values():
        SO += j
    IBB = 0
    for j in baseballYear['IBB'].values():
        IBB += j
    baseballYearTotal.append([i,G,AB,R,H,Double,Triple,HR,RBI,BB,SO,IBB])
newBaseball = pd.DataFrame(baseballYearTotal, columns=['yearID','G','AB','R','H','Double','Triple','HR','RBI','BB','SO','IBB'])
newBaseball

Now that we have our statistics divided up by years, we will assess the overall trend in home runs over the last two decades of seasons and move on to look for significant variables and collinearity.

In [ ]:
newBaseball.plot.bar(x='yearID', y='HR', rot=90)

It appears that there is a general upwards trend in home runs hit each year with a steep decrease in 2020 because COVID-19 shortened the season dramatically. We expect to see all of the predictors values to reflect this decrease proportionately. In 2021, it is expected that the number of home runs will return to the rising trend over the last decade. In the baseball community, this has become known as the "live ball era" because of the upwards trend in home runs. 

Now that we have our data laid out in terms of total stats per year, we can continue.

In [ ]:
mod = smf.ols(formula='HR ~ 1 + yearID + G + AB + R + H + Double + Triple + RBI + BB + SO + IBB', data = newBaseball)

In [ ]:
res = mod.fit()
res.summary()

We decided to do a VIF analysis to determine collinearity in the predictors and found that many of the variables had a VIF score of above ten. This indicates a large amount of collinearity. We will not drop the predictors because of this but it is important to keep in mind as we head into our next step of subset selection. 

In [ ]:
vif = pd.DataFrame()
vif['X'] = newBaseball.columns
vif['vif'] = [variance_inflation_factor(newBaseball.values, i) for i in range(len(newBaseball.columns))]
vif

## Subset Selection

After several attempts using different subset selection models, we've decided that with the number of different predictors we have Forward Subset Selection was the best one to use. With that, we are going to start by using the Forward Subset Selection algorithm to determine what the best subset (combination) of the predictors are. We will perform several iteratations of the process below to find the least complex model with the lowest BIC. This will not necessarily be the best model, however.

In [ ]:
newBaseball.head()

### Predictor 1

In [ ]:
metric_store = [[],[]]
for i, combination in enumerate(combinations([0,1,2,3,4,5,6,8,9,10,11],1), 1):
    x_data = sm.add_constant(newBaseball.iloc[:,list(combination)])
    mod  = sm.OLS(newBaseball.HR, x_data).fit()
    metric_store[0].append(list(combination))
    metric_store[1].append(mod.bic)

In [ ]:
metric_store

In [ ]:
metric_store[0][np.argmin(metric_store[1])], np.min(metric_store[1])

### Predictor 2

In [ ]:
metric_store = [[],[]]
for i, combination in enumerate(combinations([0,1,2,3,4,5,6,9,10,11],1), 1):
    x_data = sm.add_constant(newBaseball.iloc[:,[8] + list(combination)])
    mod  = sm.OLS(newBaseball.HR, x_data).fit()
    metric_store[0].append(list(combination))
    metric_store[1].append(mod.bic)

In [ ]:
metric_store[0][np.argmin(metric_store[1])], np.min(metric_store[1])

### Predictor 3

In [ ]:
metric_store = [[],[]]
for i, combination in enumerate(combinations([0,1,2,3,4,5,6,9,10],1), 1):
    x_data = sm.add_constant(newBaseball.iloc[:,[8,11] + list(combination)])
    mod  = sm.OLS(newBaseball.HR, x_data).fit()
    metric_store[0].append(list(combination))
    metric_store[1].append(mod.bic)

In [ ]:
metric_store[0][np.argmin(metric_store[1])], np.min(metric_store[1])

### Predictor 4

In [ ]:
metric_store = [[],[]]
for i, combination in enumerate(combinations([0,1,2,4,5,6,9,10],1), 1):
    x_data = sm.add_constant(newBaseball.iloc[:,[8,11,3] + list(combination)])
    mod  = sm.OLS(newBaseball.HR, x_data).fit()
    metric_store[0].append(list(combination))
    metric_store[1].append(mod.bic)

In [ ]:
metric_store[0][np.argmin(metric_store[1])], np.min(metric_store[1])

### Predictor 5

In [ ]:
metric_store = [[],[]]
for i, combination in enumerate(combinations([0,1,2,4,6,9,10],1), 1):
    x_data = sm.add_constant(newBaseball.iloc[:,[8,11,3,5] + list(combination)])
    mod  = sm.OLS(newBaseball.HR, x_data).fit()
    metric_store[0].append(list(combination))
    metric_store[1].append(mod.bic)

In [ ]:
metric_store[0][np.argmin(metric_store[1])], np.min(metric_store[1])

### Predictor 6

In [ ]:
metric_store = [[],[]]
for i, combination in enumerate(combinations([0,1,2,4,6,9],1), 1):
    x_data = sm.add_constant(newBaseball.iloc[:,[8,11,3,5,10] + list(combination)])
    mod  = sm.OLS(newBaseball.HR, x_data).fit()
    metric_store[0].append(list(combination))
    metric_store[1].append(mod.bic)

In [ ]:
metric_store[0][np.argmin(metric_store[1])], np.min(metric_store[1])

### Predictor 7

In [ ]:
metric_store = [[],[]]
for i, combination in enumerate(combinations([0,1,2,4,6],1), 1):
    x_data = sm.add_constant(newBaseball.iloc[:,[8,11,3,5,10,9] + list(combination)])
    mod  = sm.OLS(newBaseball.HR, x_data).fit()
    metric_store[0].append(list(combination))
    metric_store[1].append(mod.bic)

In [ ]:
metric_store[0][np.argmin(metric_store[1])], np.min(metric_store[1])

In [ ]:
newBaseball.head()

According to our Forward Subset Selection, we have 6 chosen predictors. We have stopped at 6 since our testing for Predictor 7 has resulted in a higher BIC than the testing for Predictor 6.

Thus, our 6 predictors are as follows:


| Num | ID     | Name                       |
|-----|--------|----------------------------|
|  3  | R      | Runs                       |
|  5  | Double | Doubles                    |
|  8  | RBI    | Runs Batted In             |
|  9  | BB     | Base on Balls              |
| 10  | SO     | Strikeouts                 |
| 11  | IBB    | Intentional Walks          |

In [ ]:
mod = smf.ols(formula='HR ~ 1 + R + Double + RBI + BB + SO + IBB', data = newBaseball)
res = mod.fit()
res.summary()

In [ ]:
plt.figure(dpi = 150)
plt.plot(newBaseball['R'], newBaseball['HR'], '.', markersize=10, markeredgecolor="black", color="goldenrod")
plt.plot(newBaseball['Double'], newBaseball['HR'], '.', markersize=10, markeredgecolor="black", color="red")
plt.plot(newBaseball['RBI'], newBaseball['HR'], '.', markersize=10, markeredgecolor="black", color="green")
plt.plot(newBaseball['BB'], newBaseball['HR'], '.', markersize=10, markeredgecolor="black", color="blue")
plt.plot(newBaseball['SO'], newBaseball['HR'], '.', markersize=10, markeredgecolor="black", color="pink")
plt.plot(newBaseball['IBB'], newBaseball['HR'], '.', markersize=10, markeredgecolor="black", color="purple")
plt.ylabel("Home Runs")
plt.grid()
plt.show()

This is a plot of all of the predictors plotted with home run counts. We see a slight positive correlation in multiple predictors. While not useful for specifically predicting home run count, this figure is a good visual to see any obvious relationships between the predictors and the response.

## Leave One Out Cross Validation

We are now going to proceed with using Leave One Out Cross Validation ('LOOCV') to be testing which polynomial degree our model should be fit to, and then testing the accuracy of the model with that polynomial degree with LOOCV.

In [ ]:
loocv = LeaveOneOut()
regr = skl_lm.LinearRegression()
M = 16
pred = [3,5,8,9,10,11]
loocv_mse = []

for i in range(M):
    predMSE = []
    for j in pred:
        x_poly = PolynomialFeatures(i).fit_transform(newBaseball.iloc[:,j].to_numpy().reshape(-1,1))
        mse = cross_val_score(regr, x_poly, newBaseball.HR, cv=loocv, scoring='neg_mean_squared_error').mean()
        predMSE.append(-mse)
    val = 0
    for j in predMSE:
        val += j
    loocv_mse.append(val)

In [ ]:
loocv_mse

In [ ]:
plt.figure(dpi=150)
plt.plot(range(M),loocv_mse,'.-',markersize=10)
plt.xlabel("Polynomial Degree"); plt.ylabel("MSE")
plt.grid()
plt.show()

Through our LOOCV test, we have determined, to our surprise, that a linear model is the best fit for the data. Now, because our dataset has few rows, we will use Leave One Out Cross Validation to determine the accuracy and error in our model. After that, we will use our hopefully accurate model to attempt to predict the home run count for the 2021 MLB season.


In [ ]:
from numpy import mean
from numpy import std
my_preds = [3,5,8,9,10,11]
X = newBaseball.iloc[:,[3,5,8,9,10,11]]
loocv = LeaveOneOut()
regr = skl_lm.LinearRegression()
scores = cross_val_score(regr, newBaseball.iloc[:,[3,5,8,9,10,11]], newBaseball.HR, scoring='neg_mean_squared_error', cv=loocv, n_jobs=-1)
print('Accuracy: %.3f (%.3f)' % (mean(-scores), std(scores)))

This is our MSE for our model.

This is a fairly accurate model overall. Next, we will predict the home run count for the 2021 season and see how close we get to the actual value.

## Testing

In [ ]:
baseball2021 = pd.read_csv("https://raw.githubusercontent.com/dswetlik/BaseballHRPrediction/master/2021Batting.csv")
baseball2021

In [ ]:
mod = skl_lm.LinearRegression()
Y_pred = mod.fit(newBaseball.iloc[:,[3,5,8,9,10,11]], newBaseball.HR).predict(baseball2021.iloc[:,[3,5,8,9,10,11]])
Y_pred[0]

Our model has predicted a home run count of 6204 for the year 2021. The actual home run count for 2021 was 5944. Our model overshot the total home runs by 260.

## Conclusion

We are incredibly happy with our findings here. While our model predicted a total home run count of 260 over the actual number, we consider that to be a relatively accurate prediction. Both Diedrich and myself considered this to be within reasonable bounds of error. Our model is extremely biased, though, as we are using a one-degree polynomial and have six predictors. Because of this, our model's flexibility is extremely low. We think it would be very interesting to see how accurate our model would be if we added 2021 to the dataset and try to predict the next season's home run count to see if the model holds up well over time.